In [14]:
from __future__ import print_function

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [16]:
from ilqr.cost import QRCost, FiniteDiffCost
from ilqr.mujoco_dynamics import MujocoDynamics
from ilqr.mujoco_controller import iLQR, RecedingHorizonController
from ilqr.examples.cartpole import CartpoleDynamics
from ilqr.dynamics import constrain
from ilqr.utils.visualization import make_video_fn

from scipy.optimize import approx_fprime

import mujoco_py
from mujoco_py import MjViewer
import os

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
def on_iteration(iteration_count, xs, us, J_opt, accepted, converged):
    J_hist.append(J_opt)
    info = "converged" if converged else ("accepted" if accepted else "failed")
    final_state = xs[-1]
    print("iteration", iteration_count, info, J_opt, final_state)

In [18]:
xml_path = os.path.join('..', 'ilqr', 'xmls', 'reacher.xml')

#STATE: [theta1, theta2, goal_x, goal_y, theta1_dot, theta2_dot, goal_x_dot, goal_y_dot]

dynamics = MujocoDynamics(xml_path, frame_skip = 2, use_multiprocessing = True)
print('dt:', dynamics.dt)

Finished loading process 31342
Finished loading process 31343
 Finished loading processFinished loading process Finished loading processFinished loading process31346Finished loading process   
3135931368
31363
31362Finished loading process

Finished loading process  31379
31382
Finished loading process 31406
dt: 0.02
Finished loading process 31395
Finished loading process 31423


In [19]:
def l(x, u, i):
    jointposx = 0.1 * np.cos(x[0])
    jointposy = 0.1 * np.sin(x[0])
    fingerposx = jointposx + 0.1 * np.cos(x[0] + x[1])
    fingerposy = jointposy + 0.1 * np.sin(x[0] + x[1])
    
    dist = np.sqrt((fingerposx - x[2]) ** 2 + (fingerposy - x[3]) ** 2)
    return 100 * (dist) + x[4] ** 2 + x[5] ** 2 + u[0] ** 2 + u[1] ** 2

cost2 = FiniteDiffCost(l, lambda x, i: l(x, [0.0, 0.0], i), 8, 2, use_multiprocessing = True)

Finished loading process Finished loading process31453 
31456Finished loading process
 Finished loading process31461Finished loading process 31472
 Finished loading process
31477
 Finished loading process31486 31499

Finished loading process Finished loading process31516
 Finished loading process31519 
31526
Finished loading process  31533Finished loading process
31546


In [20]:
N = 100
x0 = np.array([0.0, 0.0, np.random.uniform(-0.14, 0.14), np.random.uniform(-0.14, 0.14), 0.0, 0.0, 0.0, 0.0])
#us_init = np.random.uniform(-1, 1, (N, dynamics.action_size))
us_init = np.zeros((N, dynamics.action_size))
ilqr = iLQR(dynamics, cost2, N)
mpc = RecedingHorizonController(x0, ilqr)

In [21]:
t0 = time.time()
J_hist = []
controls = mpc.control(us_init, step_size = 1, initial_n_iterations = 500, subsequent_n_iterations = 100, on_iteration = on_iteration)
us = []
for i in range(30):
    print('ITERATION', i, '\n')
    if i == 29:
        us.append(next(controls)[2])
    else:
        us.append(next(controls)[1])
    
print('time', time.time() - t0)
us = np.concatenate(us)

ITERATION 0 

iteration 0 failed 2637.1158784961985 [ 0.          0.         -0.04637334 -0.08645052  0.          0.
  0.          0.        ]
iteration 1 failed 2637.1158784961985 [ 0.          0.         -0.04637334 -0.08645052  0.          0.
  0.          0.        ]
iteration 2 failed 2637.1158784961985 [ 0.          0.         -0.04637334 -0.08645052  0.          0.
  0.          0.        ]
iteration 3 accepted 1593.8960189366555 [-1.30503409 -1.0041661  -0.04637334 -0.08645052  0.01272949  0.00978354
  0.          0.        ]
iteration 4 accepted 1153.7142936926593 [-1.33335902 -2.47867485 -0.04637334 -0.08645052  0.01311793  0.02401575
  0.          0.        ]
iteration 5 accepted 994.4792812833647 [-0.93451996 -2.11258438 -0.04637334 -0.08645052  0.00971139  0.02102262
  0.          0.        ]
iteration 6 accepted 960.8519658074241 [-1.12136877 -2.14495829 -0.04637334 -0.08645052  0.0082863   0.02260368
  0.          0.        ]
iteration 7 accepted 915.2470959300188 [-1.00

iteration 54 failed 855.5735077556905 [-1.00505491e+00 -2.11624377e+00 -4.63733366e-02 -8.64505231e-02
 -7.96064371e-03  6.41954604e-04  0.00000000e+00  0.00000000e+00]
iteration 55 accepted 855.57160590065 [-1.00505489e+00 -2.11624377e+00 -4.63733366e-02 -8.64505231e-02
 -7.95471623e-03  6.46571187e-04  0.00000000e+00  0.00000000e+00]
iteration 56 failed 855.5716059006502 [-1.00505489e+00 -2.11624377e+00 -4.63733366e-02 -8.64505231e-02
 -7.95471623e-03  6.46571187e-04  0.00000000e+00  0.00000000e+00]
iteration 57 failed 855.5716059006502 [-1.00505489e+00 -2.11624377e+00 -4.63733366e-02 -8.64505231e-02
 -7.95471623e-03  6.46571187e-04  0.00000000e+00  0.00000000e+00]
iteration 58 failed 855.5716059006502 [-1.00505489e+00 -2.11624377e+00 -4.63733366e-02 -8.64505231e-02
 -7.95471623e-03  6.46571187e-04  0.00000000e+00  0.00000000e+00]
iteration 59 converged 855.5711976129566 [-1.00504852e+00 -2.11624413e+00 -4.63733366e-02 -8.64505231e-02
 -7.55619844e-03  9.43161044e-04  0.00000000e+00 

iteration 5 converged 508.4477850237072 [-1.00459639 -2.11676102 -0.04637334 -0.08645052  0.01683423 -0.02099426
  0.          0.        ]
ITERATION 9 

iteration 0 failed 468.91095739974713 [-1.00419904 -2.11725879 -0.04637334 -0.08645052  0.02288088 -0.02875699
  0.          0.        ]
iteration 1 failed 468.91095739974713 [-1.00419904 -2.11725879 -0.04637334 -0.08645052  0.02288088 -0.02875699
  0.          0.        ]
iteration 2 converged 468.9108045928624 [-1.00420047 -2.1172572  -0.04637334 -0.08645052  0.02287968 -0.02875535
  0.          0.        ]
ITERATION 10 

iteration 0 failed 431.3505215689384 [-1.00368324 -2.11790894 -0.04637334 -0.08645052  0.02882344 -0.03639348
  0.          0.        ]
iteration 1 failed 431.3505215689384 [-1.00368324 -2.11790894 -0.04637334 -0.08645052  0.02882344 -0.03639348
  0.          0.        ]
iteration 2 failed 431.3505215689384 [-1.00368324 -2.11790894 -0.04637334 -0.08645052  0.02882344 -0.03639348
  0.          0.        ]
iteration 3

iteration 0 accepted 179.30031787874105 [-1.00248864 -2.11944977 -0.04637334 -0.08645052  0.12673183 -0.16425351
  0.          0.        ]
iteration 1 failed 179.30031787874097 [-1.00248864 -2.11944977 -0.04637334 -0.08645052  0.12673183 -0.16425351
  0.          0.        ]
iteration 2 failed 179.30031787874097 [-1.00248864 -2.11944977 -0.04637334 -0.08645052  0.12673183 -0.16425351
  0.          0.        ]
iteration 3 failed 179.30031787874097 [-1.00248864 -2.11944977 -0.04637334 -0.08645052  0.12673183 -0.16425351
  0.          0.        ]
iteration 4 failed 179.30031787874097 [-1.00248864 -2.11944977 -0.04637334 -0.08645052  0.12673183 -0.16425351
  0.          0.        ]
iteration 5 accepted 179.29415854386946 [-1.00252183 -2.11940351 -0.04637334 -0.08645052  0.12667926 -0.16418083
  0.          0.        ]
iteration 6 accepted 179.2934396240103 [-1.00313039 -2.11859347 -0.04637334 -0.08645052  0.12553625 -0.16260118
  0.          0.        ]
iteration 7 failed 179.2934396240103

iteration 20 accepted 142.22428674383977 [-1.00490855 -2.11639738 -0.04637334 -0.08645052  0.02067363 -0.02664533
  0.          0.        ]
iteration 21 accepted 142.22283763906688 [-1.00490321 -2.11640534 -0.04637334 -0.08645052  0.02066506 -0.02661539
  0.          0.        ]
iteration 22 accepted 142.2213522093415 [-1.00497193 -2.11631644 -0.04637334 -0.08645052  0.0192654  -0.02516825
  0.          0.        ]
iteration 23 failed 142.22135220934157 [-1.00497193 -2.11631644 -0.04637334 -0.08645052  0.0192654  -0.02516825
  0.          0.        ]
iteration 24 failed 142.22135220934157 [-1.00497193 -2.11631644 -0.04637334 -0.08645052  0.0192654  -0.02516825
  0.          0.        ]
iteration 25 accepted 142.2209881090994 [-1.00496848 -2.11631829 -0.04637334 -0.08645052  0.0192403  -0.02515264
  0.          0.        ]
iteration 26 failed 142.22098810909947 [-1.00496848 -2.11631829 -0.04637334 -0.08645052  0.0192403  -0.02515264
  0.          0.        ]
iteration 27 failed 142.2209

iteration 6 failed 111.04331043140594 [-1.00419648 -2.11726126 -0.04637334 -0.08645052  0.05754565 -0.06903882
  0.          0.        ]
iteration 7 accepted 111.0431491047735 [-1.00419679 -2.117261   -0.04637334 -0.08645052  0.05754202 -0.06903332
  0.          0.        ]
iteration 8 converged 111.04308469214321 [-1.0042043  -2.11725326 -0.04637334 -0.08645052  0.05739768 -0.06883171
  0.          0.        ]
ITERATION 24 

iteration 0 accepted 97.4955279658629 [-1.00446983 -2.11688536 -0.04637334 -0.08645052  0.05906724 -0.06924831
  0.          0.        ]
iteration 1 accepted 97.49434127807051 [-1.00447197 -2.116883   -0.04637334 -0.08645052  0.05891034 -0.06908142
  0.          0.        ]
iteration 2 accepted 97.48483496064632 [-1.00450702 -2.11684472 -0.04637334 -0.08645052  0.05625242 -0.06630028
  0.          0.        ]
iteration 3 accepted 97.4836342895705 [-1.00450949 -2.11684202 -0.04637334 -0.08645052  0.05606181 -0.06610712
  0.          0.        ]
iteration 4 failed 9

iteration 21 accepted 73.94401724588043 [-1.00487666 -2.11631038 -0.04637334 -0.08645052  0.02686682 -0.03780809
  0.          0.        ]
iteration 22 failed 73.94401724588046 [-1.00487666 -2.11631038 -0.04637334 -0.08645052  0.02686682 -0.03780809
  0.          0.        ]
iteration 23 failed 73.94401724588046 [-1.00487666 -2.11631038 -0.04637334 -0.08645052  0.02686682 -0.03780809
  0.          0.        ]
iteration 24 failed 73.94401724588046 [-1.00487666 -2.11631038 -0.04637334 -0.08645052  0.02686682 -0.03780809
  0.          0.        ]
iteration 25 accepted 73.93678131884798 [-1.00487838 -2.11631887 -0.04637334 -0.08645052  0.02685964 -0.03779268
  0.          0.        ]
iteration 26 accepted 73.9339933142151 [-1.00490284 -2.11633257 -0.04637334 -0.08645052  0.02675837 -0.03766195
  0.          0.        ]
iteration 27 accepted 73.93385212609748 [-1.0049032  -2.11633249 -0.04637334 -0.08645052  0.02675496 -0.03765703
  0.          0.        ]
iteration 28 accepted 73.932342118

iteration 32 failed 54.786265746707016 [-1.00483631 -2.11645104 -0.04637334 -0.08645052  0.02780137 -0.03211141
  0.          0.        ]
iteration 33 failed 54.786265746707016 [-1.00483631 -2.11645104 -0.04637334 -0.08645052  0.02780137 -0.03211141
  0.          0.        ]
iteration 34 accepted 54.785409462202864 [-1.00483598 -2.11645057 -0.04637334 -0.08645052  0.02779795 -0.03210768
  0.          0.        ]
iteration 35 accepted 54.78268999723251 [-1.00483914 -2.11644331 -0.04637334 -0.08645052  0.02775181 -0.0320584
  0.          0.        ]
iteration 36 failed 54.782689997232545 [-1.00483914 -2.11644331 -0.04637334 -0.08645052  0.02775181 -0.0320584
  0.          0.        ]
iteration 37 failed 54.782689997232545 [-1.00483914 -2.11644331 -0.04637334 -0.08645052  0.02775181 -0.0320584
  0.          0.        ]
iteration 38 failed 54.782689997232545 [-1.00483914 -2.11644331 -0.04637334 -0.08645052  0.02775181 -0.0320584
  0.          0.        ]
iteration 39 accepted 54.7821721530

iteration 49 accepted 46.685362154461934 [-1.00474554 -2.11657495 -0.04637334 -0.08645052  0.02894973 -0.0368467
  0.          0.        ]
iteration 50 accepted 46.685118555229444 [-1.0047465  -2.11657375 -0.04637334 -0.08645052  0.02894344 -0.03683797
  0.          0.        ]
iteration 51 accepted 46.68497478897113 [-1.00474722 -2.11657309 -0.04637334 -0.08645052  0.0289363  -0.03682783
  0.          0.        ]
iteration 52 accepted 46.684800310204366 [-1.00476668 -2.11655378 -0.04637334 -0.08645052  0.02839023 -0.03615167
  0.          0.        ]
iteration 53 failed 46.68480031020441 [-1.00476668 -2.11655378 -0.04637334 -0.08645052  0.02839023 -0.03615167
  0.          0.        ]
iteration 54 failed 46.68480031020441 [-1.00476668 -2.11655378 -0.04637334 -0.08645052  0.02839023 -0.03615167
  0.          0.        ]
iteration 55 failed 46.68480031020441 [-1.00476668 -2.11655378 -0.04637334 -0.08645052  0.02839023 -0.03615167
  0.          0.        ]
iteration 56 failed 46.68480031

In [22]:
dynamics.set_state(x0)
print(dynamics.get_state())
video = []
for i in range(us.shape[0]):
    dynamics.step(us[i])
    video.append(dynamics.sim.render(512, 512))

[ 0.          0.         -0.04637334 -0.08645052  0.          0.
  0.          0.        ]


In [23]:
make_video_fn(video)()